In [ ]:
from math import exp, log, sqrt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt

from linear_motion_sim import Simulator, fit_ks_and_kv, fit_ka
from func_guesser import sign

In [ ]:
sim = Simulator(max_speed=20, max_accel=12, drag=0.8, noise=0.2)

In [ ]:
ks, kv = fit_ks_and_kv(sim)
ka = fit_ka(sim, ks, kv)

In [ ]:
ac = sim.sim(lambda t, x, v, a: -ks + kv * v + ka * -2)
sim.plot_curves(ac)

In [ ]:
tgt_x = -60
tgt_v = 0

vtol = 0.1
xtol = 0.5
ttol = 0.1

In [ ]:
def ratelimit(x1, t1, x0, t0, v):
    dt = t1 - t0
    up = x0 + v * dt
    dn = x0 - v * dt
    return min(max(x1, dn), up)

In [ ]:
class Drive:
    def __init__(self, alimit, vlimit, max_jerk):
        self.alimit = alimit
        self.vlimit = vlimit
        self.done = False
        self.max_jerk = max_jerk
        self.prev_t = 0


    def drive_power(self, t, x, v, a):
        if self.done:
            return 0.05 * np.random.normal()

        if abs(x - tgt_x) < xtol and abs(v - tgt_v) < vtol:
            self.done = True
            return 0
        
        dist = tgt_x - x
        v0 = sqrt(2 * self.alimit * abs(dist))
        v1 = sign(dist) * min(v0, self.vlimit)
        
        if abs(v1 - v) < vtol:
            a0 = 0
        else:
            a0 = self.alimit * sign(v1 - v)

        dt = t - self.prev_t
        self.prev_t = t
        
        a1 = ratelimit(a0, t, a, self.prev_t, self.max_jerk)
        self.prev_t = t
        print(f'{a0:.4f} {a1:.4f}')
        
        return ks + kv * v + ka * a1

In [ ]:
d = Drive(alimit=8, vlimit=12, max_jerk=700)
ac = sim.sim(d.drive_power, max_t=15)
sim.plot_curves(ac, tgt_x=tgt_x)